# DSCI 525 Web and Cloud Computing 
## Milestone 1 Tackling big data on your laptop 
Authors: Amelia Tang, Chaoran Wang, Junrong Zhu (Group 13) 

### Import Dependencies

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

### Downloading the data
1. Download the data from figshare to local computers using the figshare API and requests library.
2. Extract the zip file

In [2]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [5]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [6]:
os.remove("figshare/observed_daily_rainfall_SYD.csv")

### Combining data CSVs
1. Combine data CSVs into a single CSV using pandas.
2. When combining the CSV files, add an extra column called "model" that identifies the model. 
3. Compare run times on different machines within our team. 

In [7]:
%%time

use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
files = glob.glob('figshare/*.csv')

df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r"/([^_]*)", file)[0])
                for file in files))

df.to_csv("figshare/combined_data.csv")

CPU times: user 5min 57s, sys: 13.2 s, total: 6min 10s
Wall time: 6min 31s


#### Time Comparison Table for Combining CSVs


| Team Member  |Operating System|RAM|Processor|Is SSD| Time Taken|
| -------------| -------------- | - | --- ----| -----| --------- |
| Junrong Zhu  |macOS Monterey  |8GB|   CPU - Apple M1 chip 8-core   |  Yes | Total time 5min 57s|
| Amelia Tang  |macOS Monterey  |8GB|   CPU - 2.2 GHz Dual-Core Intel Core i7   |  Yes | Total time 10min 1s|
| Chaoran Wang | macOS Big Sur   | 16GB  | CPU - 4.2 Ghz Quad-Core Intel Core i7 | Yes | Total time 5min 27s |

***Our Observations***
We observed that computers whose CPU had more cores tended to combine the files faster. Besides, the more RAM a computer had, the less time it took to process the files. Given that our operating systems were all MacOS and we all had SSD, we did not observe how different operating systems and whether have SSD affected the speed. However, based on our research, different operating systems do impact the speed so do the specifications of SSDs.  

Sources: https://dash.harvard.edu/bitstream/handle/1/24829608/tr-09-95.pdf
<br>https://ssdsphere.com/how-does-ssd-speed-up-a-system/

#### In order to understand our data better, we performed following exploratory data analysis steps:

**Python:**
- observing and changing the `dtype` of the data
- loading the columns of interest

**R:**
- obtaining summary statistic of columns
- constructing plot of parameters of interests

## Load CSV to memory and perform a simple EDA in Python

In [8]:
df = pd.read_csv("figshare/combined_data.csv", parse_dates=True, index_col='time')

#### Changing `dtype` of the data

We firstly observed the `dtype` for each column. 

In [9]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

Then, we used `.describe()` to do a simple EDA on the combined dataset with default data types for all the columns and we loaded all the original columns. We timed the process to establish the baseline for comparisons. 

In [10]:
%%time
df.describe() # baseline

CPU times: user 7.43 s, sys: 8.22 s, total: 15.6 s
Wall time: 22.8 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


We further explored the memory consumption with different `dtype` for the numeric columns below, and we assumed that less memory usage would be likely to lead to less running time.

In [11]:
print(f"Memory usage with float64: {df[['lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2998.46 MB
Memory usage with float32: 1749.10 MB


We observed that using data type `float32` instead of `float64` would save about half of the memory, so we changed 5 numeric columns from `float64` to `float32` in the code cell below.

In [12]:
# converting dtype
colum_dtypes = {'lat_min': np.float32, 
                'lat_max': np.float32, 
                'lon_min': np.float32, 
                'lon_max': np.float32, 
                'rain (mm/day)': np.float32}

In [13]:
%%time
df_new = pd.read_csv("figshare/combined_data.csv", 
                     parse_dates=True, index_col='time', dtype=colum_dtypes)

CPU times: user 50.4 s, sys: 10.7 s, total: 1min 1s
Wall time: 1min 20s


In [14]:
# Check the columns' data type after converting
df_new.dtypes

lat_min          float32
lat_max          float32
lon_min          float32
lon_max          float32
rain (mm/day)    float32
model             object
dtype: object

In [15]:
%%time
df_new.describe() # time comparison

CPU times: user 5.38 s, sys: 2.34 s, total: 7.72 s
Wall time: 8.2 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310497e+01,-3.197765e+01,1.469057e+02,1.482150e+02,1.901173e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


#### Loading the columns of interest

Since we were mostly interested in the rainfall in mm/day, we loaded only the `time` and `rain (mm/day)` columns this time to perform the same eda step. 

In [16]:
df_subset = pd.read_csv("figshare/combined_data.csv",
                        parse_dates=True, index_col='time', 
                        usecols=['time', 'rain (mm/day)'])

df_subset.head()

,rain (mm/day)
time,
1889-01-01 12:00:00,4.244226e-13
1889-01-02 12:00:00,4.217326e-13
1889-01-03 12:00:00,4.498125e-13
1889-01-04 12:00:00,4.251282e-13
1889-01-05 12:00:00,4.270161e-13


In [17]:
%%time
df_subset.describe() # running same EDA step on the subset data

CPU times: user 2.12 s, sys: 2.25 s, total: 4.37 s
Wall time: 8.01 s


,rain (mm/day)
count,5.924854e+07
mean,1.901170e+00
std,5.585735e+00
min,-3.807373e-12
25%,3.838413e-06
50%,6.154947e-02
75%,1.020918e+00
max,4.329395e+02


### Comparison table for Python EDA timing

| Team Member  |Operating System|RAM|Processor|Is SSD| Baseline time for EDA | Time after changing `dtype`| Time for fewer columns|
| -------------| -------------- | - | --- ----| -----| --------- |  --------- |  --------- |
| Junrong Zhu  |macOS Monterey  |8GB|   CPU - Apple M1 chip 8-core   |  Yes | Total: 13.5s Wall time: 16s|  Total: 7.35s Wall time: 7.73s |  total: 3.04 s Wall time: 3.24 s
| Amelia Tang  |macOS Monterey  |8GB|   CPU - 2.2 GHz Dual-Core Intel Core i7   |  Yes | Total: 22.4s Wall time: 26.1s| Total: 13.2s Wall time: 15s | total: 4.82 s Wall time: 4.97 s 
| Chaoran Wang | macOS Big Sur   | 16GB | CPU - 4.2 Ghz Quad-Core Intel Core i7 | Yes | Total: 12.7s Wall time: 12.8s| Total: 8s Wall time: 8.06s | total: 3.53 s Wall time: 3.57 s

### Summary for Python 
- Changing `dtype` 
> After changing the `dtype` from `float64` to `float32`, the memory usage decreased by around 50% and we observed obvious decreases in total / wall time to perform the simple EDA across our team member's computers.  
- Loading on the columns needed 
> Since our main focus here was daily rainfall so we are most interested in the `rain (mm/day)` column. After extracting the subset of data, we saw significant decreases in total / wall time comparing to both Baseline time and the time after converting `dtype` across our team members' computers. 

### R Section

#### Transfer dataframe from python to R

We will run a few experiments below to see which file is suitable for us.

In [18]:
df_new = df_new.reset_index()

In [19]:
# writing feature and parquet files

df_new.to_feather("figshare/combined_data.feather")
df_new.to_parquet("figshare/combined_data.parquet")

In [20]:
%%sh
du -sh figshare/combined_data.feather

960M	figshare/combined_data.feather


In [21]:
%%sh
du -sh figshare/combined_data.parquet

353M	figshare/combined_data.parquet


We can see our feather file of the data is about 950M and parquet takes less memory with about 350M.

In [22]:
%%sh
du -sh figshare/combined_data.csv

5.6G	figshare/combined_data.csv


`.csv`file is the largest among three formats

In [23]:
# Comparing the time to read in the files

In [25]:
%load_ext rpy2.ipython

In [30]:
%%R
start_time <- Sys.time()
df_feather <- arrow::read_feather(‘figshare/combined_data.feather’)
end_time <- Sys.time()
end_time - start_time

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [ ]:
%%time
%%R -i df
start_time <- Sys.time()
df_parquet <- arrow::read_parquet(‘figshare/combined_data.parquet’)
end_time <- Sys.time()
end_time - start_time

Even though feather file is larger than the parquet file, it takes less time than the parquet file when we read the data so that we can assume feather has optimal efficiency.

#### Reasoning on choosing `Feather File`:
- Based on the experiments above, we decide to use `feather file` over `Parquet file` considering its processing time. 
- Also, `feather` requires minimal serialization, while `Pandas Exchange` approach would consume significant amount of memory and time on the serialization + deserialization process.
- Moreover, we do not want to simply use `Arrow Exchange` with the `csv` format because size of a .csv file is much larger than .feather file (5.6 GB vs. less than 1000M), thus running .csv is less efficient.

In [ ]:
%%R
suppressMessages(library(arrow))
df_feather <- arrow::read_feather('figshare/combined_data.feather')
head(df_feather)

### Simple EDA

In [ ]:
%%R
summary(df_feather)

In [ ]:
%%R
df_feather$year <- format(df_feather$time, format = "%Y")
head(df_feather, 3)

Based on the previous discussion, we are mostly interested in the `time` and `rain (mm/day)` columns, therefore, we'll perform further EDA with a focus on a subset of data.

In [ ]:
%%R
colnames(df_feather)[6] <- "rain"
head(df_feather, 3)

In [ ]:
%%R
df_feather$year <- format(df_feather$time, format = "%Y")
head(df_feather, 3)

In [ ]:
%%R
df_feather <- aggregate(rain ~ year, data = df_feather, mean)
head(df_feather)

In [ ]:
%%R
df_feather$year <- as.numeric(df_feather$year)

In [ ]:
%%R
suppressMessages(library(ggplot2))
ggplot(data = df_feather, aes(x = year, y = rain)) +
  geom_line() +
  labs(title = "Rainfall trend in Austrilia", x = "Year", y = "Rainfall")

## Challenges

1. One of the challenges we had with Q5 was the long running time. For example, initially tried to use `.info()` like what we did in other courses to obtain the `dtype` for each column. However, it took a long time and output additional information we were not particularly interested in, such as the number of the non-null values in each column. Alternatively, we used `.dtypes` to get the data type of columns and it returned the results quickly. 

2. For EDA in R, we were not able to create a very comprehensive EDA with sophisticated plots because of the size of the data. We managed to plot a simple line plot to show the trend of rain over years. Due to the large data size, we ended up aggregating it with means of rainfall by years instead of plotting the original rainfall data. The aggregation process took about 10 minutes for Chaoran and it seemed to be acceptable, but the kernels on Junrong's and Amelia's laptops died after running the cell for the plot for a long time. We could not overcome this challenge due to our limited computational power. If we could load this notebook to AWS SageMaker, we all should be able to run the plot.    

3. We had quite some discussions over which file format to choose between`feather` or `parquet` because one had the advantage on loading time and the other had the advantage on memory usage. We resolved this by running a few experiments and picked `feather` because it was faster and our main concern with the large data set here was the speed since our laptops had enough memory. 